# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
import requests

## Read and process static data

### Paths

In [2]:
path_npvm_zones = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\Verkehrszonen_Schweiz_NPVM_2017_shp.zip"
path_mobility_stations = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\mobility-stationen-und-fahrzeuge-schweiz.csv"
path_pt_jrta = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\140_JRTA_(OEV).mtx"
path_pt_ntr = r"D:\data\ttools gmbh\Projekte - Dokumente\021_MobilityStationFinder\Daten\144_NTR_(OEV).mtx"

### Read NPVM-zones with shapes

In [3]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 4.67 s
Wall time: 5.55 s


In [4]:
gdf_npvm_zones.head()

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
0,101001,1,1,Aeugst am Albis,1,None,1,ZH,3,Ländlich,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.47334 47.26128, 8.47334 47.26139, ..."
1,201001,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.42224 47.29775, 8.42282 47.29816, ..."
2,201002,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.44770 47.26794, 8.44767 47.26782, ..."
3,201003,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.43834 47.27714, 8.43814 47.27726, ..."
4,201004,2,2,Affoltern am Albis,1,None,1,ZH,1,Städtisch,261,Zürich,12031,Dietikon–Schlieren,"POLYGON ((8.45000 47.27949, 8.45007 47.27945, ..."


In [5]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

Anzahl NPVM-Zonen: 7978


In [6]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [7]:
get_npvm_zone(5301003)

,ID,ID_alt,ID_Gem,N_Gem,stg_type,N_stg_type,ID_KT,N_KT,ID_SL3,N_SL3,ID_Agglo,N_Agglo,ID_AMR,N_AMR,geometry
84,5301003,53,53,Bülach,1,None,1,ZH,1,Städtisch,261,Zürich,12032,Kloten,"POLYGON ((8.53973 47.51618, 8.54007 47.51529, ..."


### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [8]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [9]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

Anzahl Mobility Fahrzeuge: 2662


In [10]:
df_mobility_stations = df_mobility_vechicles.groupby("Stationsnummer").first().reset_index()

In [11]:
df_mobility_stations["lon"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_stations["lat"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[0])
df_mobility_stations = gpd.GeoDataFrame(df_mobility_stations, geometry=gpd.points_from_xy(df_mobility_stations.lon, df_mobility_stations.lat), crs=4326)

In [12]:
print("Anzahl Mobility Stationen: {}".format(len(df_mobility_stations)))

Anzahl Mobility Stationen: 1550


In [13]:
gdf_mobilty_stations_with_npvm_zone = gpd.sjoin(df_mobility_stations, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [14]:
print("Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_stations_with_npvm_zone)))

Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: 1549


In [15]:
gdf_mobilty_stations_with_npvm_zone.head()

,Stationsnummer,Name,geometry,ID,N_Gem
0,1006,Brugg Bahnhof,POINT (8.20942 47.48154),409501008,Brugg
719,3215,Brugg Post Neumarkt / Bahnhofstrasse,POINT (8.20757 47.48216),409501008,Brugg
1,1012,Arbon Bahnhof,POINT (9.43345 47.51032),440101010,Arbon
2,1019,Basel Vogesenstrasse,POINT (7.57483 47.56869),270101028,Basel
3,1024,Bellinzona Stazione,POINT (9.03017 46.19630),500201015,Bellinzona


In [16]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_stations_with_npvm_zone.dissolve(by="ID", aggfunc={"N_Gem": "first", "Name": lambda x: list(x), "Stationsnummer": lambda x: list(x)}).reset_index()

In [17]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Standort: 1311


In [256]:
gdf_npvm_zones_with_mobility_station[gdf_npvm_zones_with_mobility_station.N_Gem == "Samedan"]

,ID,geometry,N_Gem,Name,Stationsnummer
911,378601001,POINT (9.87270 46.53347),Samedan,[Samedan Bahnhof / via Retica],[4906]


### Read PT-skims

In [19]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

CPU times: total: 8.36 s
Wall time: 8.58 s


In [20]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

CPU times: total: 8.2 s
Wall time: 8.29 s


In [21]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [22]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [23]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [24]:
get_jrta(378601001, 35101026)

286.0875843780647

In [25]:
get_ntr(378601001, 35101026)

3.000000000000001

## Execute query

### Define origin and destination and assign NPVM-Zone

In [203]:
orig_lon_lat = (9.533490, 46.848940)
dest_lon_lat = (9.557410, 46.338920)

In [204]:
orig_point = Point(orig_lon_lat[0], orig_lon_lat[1])
dest_point = Point(dest_lon_lat[0], dest_lon_lat[1])

In [205]:
gdf_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
gdf_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [206]:
gdf_orig_with_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
gdf_dest_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]

In [207]:
gdf_orig_with_zone

,ID,N_Gem,geometry
0,390101022,Chur,POINT (9.53349 46.84894)


In [208]:
gdf_dest_zone

,ID,N_Gem,geometry
0,390101022,Chur,POINT (9.53349 46.84894)


In [209]:
orig_zone_id = gdf_orig_with_zone["ID"].item()

### Compute potential NPVM-zones with Mobility-station

In [258]:
gdf_circle_around_orig  = gpd.GeoDataFrame(geometry=gdf_orig_with_zone.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
gdf_circle_around_dest = gpd.GeoDataFrame(geometry=gdf_dest_zone.to_crs(2026).buffer(100*1000).to_crs(4326), crs="EPSG:4326")

In [259]:
gdf_circle_around_orig

,geometry
0,"POLYGON ((9.51597 46.81729, 9.51154 46.81862, ..."


In [260]:
gdf_circle_around_dest

,geometry
0,"POLYGON ((9.18861 46.21851, 9.10154 46.24454, ..."


In [268]:
gdf_npvm_zones_with_mobility_station_in_orig_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_orig)
gdf_npvm_zones_with_mobility_station_in_dest_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_dest)

In [269]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_orig_circle)))
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_dest_circle)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: 6
Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: 118


In [270]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

,ID,geometry,N_Gem,Name,Stationsnummer,index_right
914,390101008,POINT (9.51982 46.85790),Chur,[Chur Kurfirstenstrasse],[4605],0
915,390101009,POINT (9.51552 46.84929),Chur,[Chur Ringstrasse / sinergia],[4911],0
916,390101015,POINT (9.52621 46.86157),Chur,[Chur Lacuna],[1048],0
917,390101016,POINT (9.52944 46.85445),Chur,[Chur Bahnhof],[2842],0
918,390101020,POINT (9.53439 46.85223),Chur,[Chur Obere Quader],[4102],0


In [274]:
gdf_npvm_zones_with_mobility_station_in_dest_circle[gdf_npvm_zones_with_mobility_station_in_dest_circle.N_Gem == "Samedan"]

,ID,geometry,N_Gem,Name,Stationsnummer,index_right
911,378601001,POINT (9.87270 46.53347),Samedan,[Samedan Bahnhof / via Retica],[4906],0


In [275]:
gdf_npvm_zones_with_potential_mobility_station = pd.concat([gdf_npvm_zones_with_mobility_station_in_orig_circle, gdf_npvm_zones_with_mobility_station_in_dest_circle]).groupby("ID").first().reset_index().drop(["index_right"], axis=1)

In [276]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: {}".format(len(gdf_npvm_zones_with_potential_mobility_station)))

Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: 118


In [277]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem,Name,Stationsnummer
0,11201006,POINT (8.82323 47.27050),Bubikon,[Bubikon Bahnhof],[2295]
1,11301004,POINT (8.85052 47.26187),Dürnten,[Rüti (ZH) Kirchenrainstrasse],[2946]
2,11701006,POINT (8.83961 47.30016),Hinwil,[Hinwil Bahnhof],[2015]
3,11801004,POINT (8.85066 47.25669),Rüti (ZH),[Rüti (ZH) Gemeindehaus],[3112]
4,12001004,POINT (8.91272 47.27224),Wald (ZH),[Wald (ZH) Bahnhof],[1636]


In [278]:
gdf_npvm_zones_with_potential_mobility_station[gdf_npvm_zones_with_potential_mobility_station.ID == 30601007]

,ID,geometry,N_Gem,Name,Stationsnummer


### Get Mobility travel time and distance from ORMS

In [279]:
list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())

In [280]:
list_npvm_zones_with_potential_mobility_station[:3]

[(0, 11201006, <POINT (8.823 47.27)>, 'Bubikon', list(['Bubikon Bahnhof']), list([2295])),
 (1, 11301004, <POINT (8.851 47.262)>, 'Dürnten', list(['Rüti (ZH) Kirchenrainstrasse']), list([2946])),
 (2, 11701006, <POINT (8.84 47.3)>, 'Hinwil', list(['Hinwil Bahnhof']), list([2015]))]

In [281]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
    center = pot_mob_st[2].centroid
    coords_str += ";{},{}".format(center.x, center.y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [282]:
%time res = requests.get(url).json()

CPU times: total: 15.6 ms
Wall time: 251 ms


In [283]:
road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}

In [284]:
print("Anzahl Distanzen: {}".format(len(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))
print("Anzahl Reisezeiten: {}".format(len(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))

Anzahl Distanzen: 118
Anzahl Reisezeiten: 118


In [285]:
gdf_npvm_zones_with_potential_mobility_station.head()

,ID,geometry,N_Gem,Name,Stationsnummer
0,11201006,POINT (8.82323 47.27050),Bubikon,[Bubikon Bahnhof],[2295]
1,11301004,POINT (8.85052 47.26187),Dürnten,[Rüti (ZH) Kirchenrainstrasse],[2946]
2,11701006,POINT (8.83961 47.30016),Hinwil,[Hinwil Bahnhof],[2015]
3,11801004,POINT (8.85066 47.25669),Rüti (ZH),[Rüti (ZH) Gemeindehaus],[3112]
4,12001004,POINT (8.91272 47.27224),Wald (ZH),[Wald (ZH) Bahnhof],[1636]


In [286]:
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

In [287]:
pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0

gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_distances, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_durations, on=["ID"])


In [288]:
gdf_npvm_zones_with_potential_mobility_station_with_data.head()

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min
0,11201006,POINT (8.82323 47.27050),Bubikon,[Bubikon Bahnhof],[2295],199.7515,168.423333
1,11301004,POINT (8.85052 47.26187),Dürnten,[Rüti (ZH) Kirchenrainstrasse],[2946],198.7522,167.966667
2,11701006,POINT (8.83961 47.30016),Hinwil,[Hinwil Bahnhof],[2015],204.4139,171.521667
3,11801004,POINT (8.85066 47.25669),Rüti (ZH),[Rüti (ZH) Gemeindehaus],[3112],198.2098,166.698333
4,12001004,POINT (8.91272 47.27224),Wald (ZH),[Wald (ZH) Bahnhof],[1636],193.6107,165.263333


### Get pt skims from origin to potential Mobility stations

In [289]:
OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

In [290]:
zone_ids_list = [x.item() for x in gdf_npvm_zones_with_potential_mobility_station[["ID"]].values]
jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]

In [291]:
pd_jrtas = pd.DataFrame(jrta_list, columns=["ID", OEV_JRTA_VON_START_MIN])
pd_ntrs = pd.DataFrame(ntr_list, columns=["ID", OEV_NTR_VON_START])

In [292]:
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_jrtas, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_ntrs, on=["ID"])

In [293]:
print(len(gdf_npvm_zones_with_potential_mobility_station_with_data))

118


In [294]:
KOSTEN_CHF = "Kosten_CHF"

In [295]:
CHF_PER_KM_MOBILITY = 0.75
MIN_PER_TRANSFER = 20.0
FILTER_FACTOR = 1.05

In [296]:
def calc_costs(vtt_chf_per_h=20):
    gdf_npvm_zones_with_potential_mobility_station_with_data[KOSTEN_CHF] = \
        CHF_PER_KM_MOBILITY * gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_DISTANZ_BIS_ZIEL_KM] + \
        (gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_ZEIT_BIS_ZIEL_MIN] + gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_JRTA_VON_START_MIN] + 
        gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_NTR_VON_START] * MIN_PER_TRANSFER) / 60.0 * vtt_chf_per_h
    return gdf_npvm_zones_with_potential_mobility_station_with_data.sort_values(by=KOSTEN_CHF, ascending=True)

In [297]:
gdf_npvm_zones_with_potential_mobility_station_with_data = calc_costs(20)

In [298]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Samedan"]

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF
101,378601001,POINT (9.87270 46.53347),Samedan,[Samedan Bahnhof / via Retica],[4906],41.7546,43.146667,138.861433,1.003047,98.672298


In [299]:
def get_best_mobility_stations_per_vtt(vtt_chf_per_h):
    df_tmp = calc_costs(vtt_chf_per_h=vtt_chf_per_h)
    min_cost = df_tmp[KOSTEN_CHF].min()
    df_tmp = df_tmp[df_tmp[KOSTEN_CHF] <= min_cost * FILTER_FACTOR]
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_tmp[["ID", KOSTEN_CHF]], on="ID").sort_values(by=KOSTEN_CHF)

In [300]:
get_best_mobility_stations_per_vtt(0)

,Stationsnummer,Name,geometry,ID,N_Gem,Kosten_CHF
0,5905,St. Moritz Bahnhof,POINT (9.84740 46.49881),378701007,St. Moritz,27.060975


In [301]:
best_mobility_stations_per_vtt = {vtt: get_best_mobility_stations_per_vtt(vtt) for vtt in range(0, 101)}

In [302]:
best_mobility_stations_per_vtt[57]

,Stationsnummer,Name,geometry,ID,N_Gem,Kosten_CHF
0,2842,Chur Bahnhof,POINT (9.52944 46.85445),390101016,Chur,190.338742
1,4102,Chur Obere Quader,POINT (9.53439 46.85223),390101020,Chur,192.734548


In [303]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Steffisburg"]

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF


In [304]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Thun"]

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF


In [305]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Bern"].head(50)

,ID,geometry,N_Gem,Name,Stationsnummer,MIV_Distanz_bis_Ziel_km,MIV_Zeit_bis_Ziel_min,OEV_JRTA_von_Start_min,OEV_NTR_von_Start,Kosten_CHF


### Visualize situation on map

In [306]:
bounds = gdf_npvm_zones_with_potential_mobility_station_with_data.total_bounds
bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
bounds

[[46.323007, 8.60646], [47.51032, 10.28596]]

In [307]:
best_mobility_stations_per_vtt[11]

,Stationsnummer,Name,geometry,ID,N_Gem,Kosten_CHF
1,5905,St. Moritz Bahnhof,POINT (9.84740 46.49881),378701007,St. Moritz,65.496684
0,4906,Samedan Bahnhof / via Retica,POINT (9.87270 46.53347),378601001,Samedan,68.361941


In [308]:
def show_best_mobility_stations_per_vtt(vtt):
    geo_data = GeoData(geo_dataframe=best_mobility_stations_per_vtt[vtt],
        style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Mobility-Stationen')
    for l in m.layers:
        if type(l) == GeoData:
            m.remove_layer(l)
    m.add_layer(geo_data)

In [309]:
def on_slider_changed(event):
    show_best_mobility_stations_per_vtt(event["new"])

In [310]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, Marker, WidgetControl
from ipywidgets import IntSlider
import geopandas
import json

vtt_slider = IntSlider(description='Zeitkosten', min=0, max=100, value=20)
vtt_slider.observe(on_slider_changed, names='value')

widget_control = WidgetControl(widget=vtt_slider, position='topright')

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap, scroll_wheel_zoom=True)
m.add_control(widget_control)

m.fit_bounds(bounds)
show_best_mobility_stations_per_vtt(vtt_slider.value)
m.add_control(FullScreenControl())

m.layout.width = '100%'
m.layout.height = '500px'

orig_marker = Marker(location=(orig_lon_lat[1], orig_lon_lat[0]) , draggable=False)
dest_marker = Marker(location=(dest_lon_lat[1], dest_lon_lat[0]) , draggable=False)

m.add_layer(orig_marker)
m.add_layer(dest_marker)

m

Map(center=[52.3, 8.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…